In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pdfsss/pdf2.pdf
/kaggle/input/pdfsss/pdf1.pdf
/kaggle/input/pdfsss/pdf3.pdf


In [2]:
pip install pdfplumber pandas openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 59.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pdfplumber
import pandas as pd
import os

# Function to extract all text from a PDF and save it to CSV
def convert_pdf_to_text_csv(pdf_path, output_dir):
    """
    Extracts all text from a PDF file and saves it as a CSV file.
    
    Parameters:
    - pdf_path: Path to the PDF file
    - output_dir: Directory to save the output CSV file
    
    Returns:
    - output_csv: Path to the generated CSV file
    """
    output_csv = os.path.join(output_dir, os.path.basename(pdf_path).replace(".pdf", ".csv"))

    # Initialize an empty list to store the lines of text
    extracted_text = []

    try:
        # Open the PDF file using pdfplumber
        with pdfplumber.open(pdf_path) as pdf:
            # Iterate through all the pages in the PDF
            for page in pdf.pages:
                text = page.extract_text()
                if text:
                    # Split the extracted text into lines and append to the list
                    lines = text.split('\n')
                    extracted_text.extend(lines)
        
        # Write the extracted text into a CSV file
        with open(output_csv, 'w', encoding='utf-8') as f:
            for line in extracted_text:
                # Write each line of text as a new row in the CSV
                f.write(f'"{line.strip()}"\n')

        print(f"CSV {output_csv} created successfully from {pdf_path}.")
    except Exception as e:
        print(f"Error while converting {pdf_path} to CSV: {e}")

    return output_csv

# Function to import CSV files into Excel
def import_csvs_to_excel(csv_files, excel_path):
    """
    Imports CSV files into an Excel workbook, each CSV in a separate sheet.
    
    Parameters:
    - csv_files: List of paths to the CSV files
    - excel_path: Path to the Excel file
    """
    # Create an Excel writer object
    with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
        # Loop through each CSV file
        for csv_file in csv_files:
            try:
                # Read the CSV file as a pandas DataFrame
                df = pd.read_csv(csv_file, header=None, names=["Text"])

                # Check if the DataFrame is empty
                if df.empty:
                    print(f"Warning: {csv_file} is empty or cannot be parsed.")
                else:
                    # Generate a sheet name based on the file name (without extension)
                    sheet_name = os.path.basename(csv_file).replace('.csv', '')

                    # Write the DataFrame to a new sheet in the Excel workbook
                    df.to_excel(writer, sheet_name=sheet_name, index=False)
                    print(f"Data from {csv_file} has been imported into Excel sheet '{sheet_name}'.")
            except pd.errors.EmptyDataError:
                print(f"Error: {csv_file} is empty or has no data to parse.")
            except Exception as e:
                print(f"Error while importing {csv_file} to Excel: {e}")

    print(f"All CSV files have been imported into {excel_path}")

# Main function to automate PDF to Excel conversion
def automate_pdf_to_excel(pdf_files, output_directory, excel_file):
    """
    Automates the process of converting PDFs to CSV (with all text) and importing them into Excel.
    
    Parameters:
    - pdf_files: List of paths to the PDF files
    - output_directory: Directory to save the converted CSVs
    - excel_file: Path to the final Excel file
    """
    # List to store paths of converted CSV files
    csv_files = []

    # Convert each PDF to CSV (extracting all text) and store the CSV paths
    for pdf_file in pdf_files:
        csv_file = convert_pdf_to_text_csv(pdf_file, output_directory)
        csv_files.append(csv_file)

    # Import all CSV files into the Excel file
    import_csvs_to_excel(csv_files, excel_file)

# Paths to the PDF files (replace with your actual file paths)
pdf_files = [
    "/kaggle/input/pdfsss/pdf1.pdf",
    "/kaggle/input/pdfsss/pdf2.pdf",
    "/kaggle/input/pdfsss/pdf3.pdf"
]

# Directory where the converted CSVs will be saved
output_directory = "/kaggle/working/"

# Path to the final Excel file where the data will be stored
excel_file = "/kaggle/working/imported_data.xlsx"

# Run the automation process to extract text from PDFs and import to Excel
automate_pdf_to_excel(pdf_files, output_directory, excel_file)

CSV /kaggle/working/pdf1.csv created successfully from /kaggle/input/pdfsss/pdf1.pdf.
CSV /kaggle/working/pdf2.csv created successfully from /kaggle/input/pdfsss/pdf2.pdf.
CSV /kaggle/working/pdf3.csv created successfully from /kaggle/input/pdfsss/pdf3.pdf.
Data from /kaggle/working/pdf1.csv has been imported into Excel sheet 'pdf1'.
Data from /kaggle/working/pdf2.csv has been imported into Excel sheet 'pdf2'.
Data from /kaggle/working/pdf3.csv has been imported into Excel sheet 'pdf3'.
All CSV files have been imported into /kaggle/working/imported_data.xlsx
